#### imports

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import os

%load_ext autoreload
%autoreload 2

c:\Users\marco\Learning\pybit-trading-bot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### config

In [2]:
WINDOW_SIZE = 128
HIDDEN_SIZES = [128, 128, 128, 128, 256]
CONFIDENCE_THRESHOLDS = [0.2]
BATCH_SIZE = 2048
DROPOUT_RATE = 0.00
RESIDUAL_CONNECTIONS = True
NUM_EPOCHS = 100000

#### weights and biases

In [3]:
import wandb

wandb.init(project="bybit-trading-bot")

wandb.config.window_size = WINDOW_SIZE
wandb.config.hidden_sizes = HIDDEN_SIZES
wandb.config.batch_size = BATCH_SIZE
wandb.config.confidence_thresholds = CONFIDENCE_THRESHOLDS
wandb.config.num_epochs = NUM_EPOCHS
wandb.config.residual_connections = RESIDUAL_CONNECTIONS
wandb.config.dropout_rate = DROPOUT_RATE

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rosinusserrano (rosinusserrano-personal). Use `wandb login --relogin` to force relogin


#### preprocess data

In [9]:
from data_handling.preprocess import rates_of_change_regression, make_train_and_test, get_available_symbols

start_time_for_testing = datetime.now() - timedelta(days=60)

Xtrain, Ytrain, Xtest, Ytest = make_train_and_test(
    rates_of_change_regression, {'window_size': WINDOW_SIZE},
    start_testing_from=start_time_for_testing,
    verbose=True)

Ytest_dict = {}
Xtest_dict = {}
for filename in get_available_symbols(whole_filenames=True):
    symbol = filename.split('-')[0]
    _, _, X_tmp, Y_tmp = make_train_and_test(rates_of_change_regression,
                                             {'window_size': WINDOW_SIZE},
                                             start_time_for_testing,
                                             filename,
                                             verbose=True)
    Xtest_dict[symbol] = X_tmp
    Ytest_dict[symbol] = Y_tmp

Symbols: ['ADAUSDT-240.csv', 'APTUSDT-240.csv', 'AVAXUSDT-240.csv', 'BANANAUSDT-240.csv', 'BATUSDT-240.csv', 'BLZUSDT-240.csv', 'BNBUSDT-240.csv', 'BRETTUSDT-240.csv', 'BSVUSDT-240.csv', 'BTCUSDT-240.csv', 'C98USDT-240.csv', 'CEEKUSDT-240.csv', 'DOGEUSDT-240.csv', 'ETHUSDT-240.csv', 'LINKUSDT-240.csv', 'MEMEUSDT-240.csv', 'MNTUSDT-240.csv', 'ONDOUSDT-240.csv', 'OPUSDT-240.csv', 'POPCATUSDT-240.csv', 'SOLUSDT-240.csv', 'SUIUSDT-240.csv', 'TONUSDT-240.csv', 'WIFUSDT-240.csv', 'XRPUSDT-240.csv', 'ZENUSDT-240.csv', 'ZETAUSDT-240.csv']
Train data: 113697 entries
	last: 2024-06-19 10:00:00
Test data: 9576 entries
	first: 2024-06-19 14:00:00
Train data: (torch.Size([113569, 128]), torch.Size([113569]))
Test data: (torch.Size([9448, 128]), torch.Size([9448]))
Symbols: ['ADAUSDT-240.csv']
Train data: 7142 entries
	last: 2024-06-19 10:00:00
Test data: 361 entries
	first: 2024-06-19 14:00:00
Train data: (torch.Size([7014, 128]), torch.Size([7014]))
Test data: (torch.Size([233, 128]), torch.Size([

#### dataset and model class

In [10]:
import torch
from torch import nn
import torch.functional as F
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(torch.tensor(Xtrain), torch.tensor(Ytrain))
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)


class TradingNN(nn.Module):

    def __init__(self):
        super(TradingNN, self).__init__()
        self.fc_input = nn.Linear(WINDOW_SIZE, HIDDEN_SIZES[0])

        self.hidden_layers = nn.ModuleList()
        for i in range(len(HIDDEN_SIZES) - 1):
            self.hidden_layers.append(
                nn.Linear(HIDDEN_SIZES[i], HIDDEN_SIZES[i + 1]))

        self.fc_mean = nn.Linear(HIDDEN_SIZES[-1], 1)
        self.fc_logvar = nn.Linear(HIDDEN_SIZES[-1], 1)

        self.dropout = nn.Dropout(DROPOUT_RATE)

        self.activation = nn.SiLU()

    def forward(self, x):
        if RESIDUAL_CONNECTIONS and WINDOW_SIZE == HIDDEN_SIZES[0]:
            x = self.activation(self.fc_input(x)) + x
        else:
            x = self.activation(self.fc_input(x))

        if DROPOUT_RATE > 0:
            x = self.dropout(x)

        for i, layer in enumerate(self.hidden_layers):
            if RESIDUAL_CONNECTIONS and HIDDEN_SIZES[i] == HIDDEN_SIZES[i + 1]:
                x = self.activation(layer(x)) + x
            else:
                x = self.activation(layer(x))

            if DROPOUT_RATE > 0:
                x = self.dropout(x)

        mean = self.fc_mean(x)
        logvar = self.fc_logvar(x)

        z = torch.randn_like(mean)

        return (mean + z * torch.exp(logvar * 0.5)).squeeze(), mean, logvar


C:\Users\marco\AppData\Local\Temp\ipykernel_28788\1595941175.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset = TensorDataset(torch.tensor(Xtrain), torch.tensor(Ytrain))


#### test run

In [11]:
def test_run(model, Xtest, Ytest, threshold, dirname, filename, fee: float = 0.0005):
    if not os.path.exists(dirname):
        os.makedirs(dirname)
        
    file = open(os.path.join(dirname, filename), "w")

    money = torch.tensor(1, dtype=torch.float32)

    always_long_money = torch.tensor(1, dtype=torch.float32)

    correct_rate = 0

    with torch.no_grad():
        for i in range(len(Xtest)):
            Xtest_ = torch.tensor(Xtest[i])
            Ytest_ = torch.tensor(Ytest[i])

            _, mean, logvar = model(torch.tensor(Xtest_).float())
            std = torch.exp(logvar * 0.5)

            if std / torch.abs(mean) > threshold:
                correct = (torch.sign(mean[0]) != torch.sign(Ytest_)).item()
                correct_str = "Correct" if correct else "Incorrect"
                if correct:
                    correct_rate += 1 / len(Xtest)
                    
                file.write(
                    f"[{i} | {correct_str} | {(correct_rate * 100):.1f}%] Model is not confident enough. Mean {mean} Std {std} Ytest {Ytest_}\n"
                )
                continue
            
            correct = (torch.sign(mean[0]) == torch.sign(Ytest_)).item()
            correct_str = "Correct" if correct else "Incorrect"
            if correct:
                correct_rate += 1 / len(Xtest)

            money += money * Ytest_ * torch.sign(mean[0])
            money -= fee * money

            always_long_money += always_long_money * Ytest_
            always_long_money -= fee * always_long_money

            if mean > fee:
                file.write(
                    f"[{i} | {correct_str} | {(correct_rate * 100):.1f}%] Buying. Mean {mean} Std {std} Ytest {Ytest_}. Money is {money}\n"
                )

            elif mean < -fee:
                file.write(
                    f"[{i} | {correct_str} | {(correct_rate * 100):.1f}%] Selling. Mean {mean} Std {std} Ytest {Ytest_}. Money is {money}\n"
                )
    
    days_count = len(Xtest) / 6
    return money, days_count, always_long_money, correct_rate

#### train loop

In [12]:
model = TradingNN()
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

best_sum_of_logs = 0

for epoch in range(NUM_EPOCHS):
    for X_batch, Y_batch in tqdm(dataloader, leave=False):
        X_batch = X_batch
        optimizer.zero_grad()
        Y_pred, mean, std = model(X_batch)
        loss = criterion(Y_pred, Y_batch)
        loss.backward()
        optimizer.step()

    Ypred_test, _, _ = model(Xtest)
    loss_test = criterion(Ypred_test, Ytest)

    wandb.log({"loss": {
        "train": loss.item(),
        "test": loss_test
    }},
              commit=False)

    if (epoch + 1) % 100 == 0 or epoch == 0:
        print(f"Epoch {epoch+1} Loss: {loss.item()}")

        model.eval()

        sum_of_logs = {}

        for symbol in Xtest_dict.keys():

            confidence_money_values = {}

            for confidence_threshold in CONFIDENCE_THRESHOLDS:
                money, days, _, correct_rate = test_run(
                    model,
                    Xtest_dict[symbol],
                    Ytest_dict[symbol],
                    confidence_threshold,
                    f"output/{symbol.split('-')[0]}",
                    f"{epoch+1}.txt",
                    fee=0)

                if (epoch + 1) % 500 == 0 or epoch == 0:
                    print(symbol, confidence_threshold, money, int(days),
                          "days",
                          f"{(correct_rate * 100):.2f}% correct guesses")

                if confidence_threshold in sum_of_logs:
                    sum_of_logs[confidence_threshold] += torch.log(money)
                else:
                    sum_of_logs[confidence_threshold] = torch.log(money)
                confidence_money_values[confidence_threshold] = money.item()

            wandb.log({symbol: confidence_money_values}, commit=False)

        wandb.log({"sum_of_logs": sum_of_logs}, commit=False)

        for conf_thresh, logmoney in sum_of_logs.items():
            if logmoney > best_sum_of_logs:
                best_sum_of_logs = logmoney
                print(f"New best sum of logs {best_sum_of_logs}")
                torch.save(
                    model.state_dict(),
                    f"models/VariationalRatesOfChange_{wandb.run.name}_confidence-{conf_thresh}_BestSumOfLogs-{logmoney:.2f}.pth"
                )

        model.train()

    wandb.log({"epoch": epoch + 1})

C:\Users\marco\AppData\Local\Temp\ipykernel_28788\1993733045.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xtest_ = torch.tensor(Xtest[i])
C:\Users\marco\AppData\Local\Temp\ipykernel_28788\1993733045.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Ytest_ = torch.tensor(Ytest[i])
C:\Users\marco\AppData\Local\Temp\ipykernel_28788\1993733045.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _, mean, logvar = model(torch.tensor(Xtest_).float())


Epoch 1 Loss: 0.0007220716215670109
ADAUSDT 0.2 tensor(1.0609) 38 days 51.07% correct guesses
APTUSDT 0.2 tensor(0.9449) 39 days 50.43% correct guesses
AVAXUSDT 0.2 tensor(1.1272) 38 days 53.65% correct guesses
BANANAUSDT 0.2 tensor(0.5344) 8 days 49.06% correct guesses
BATUSDT 0.2 tensor(0.9958) 38 days 48.07% correct guesses
BLZUSDT 0.2 tensor(1.1643) 38 days 50.64% correct guesses
BNBUSDT 0.2 tensor(0.9274) 38 days 46.78% correct guesses
BRETTUSDT 0.2 tensor(1.1350) 38 days 54.08% correct guesses
BSVUSDT 0.2 tensor(0.7802) 38 days 49.79% correct guesses
BTCUSDT 0.2 tensor(0.9093) 38 days 46.78% correct guesses
C98USDT 0.2 tensor(1.2316) 39 days 50.00% correct guesses
CEEKUSDT 0.2 tensor(1.0980) 38 days 52.79% correct guesses
DOGEUSDT 0.2 tensor(0.8954) 39 days 47.01% correct guesses
ETHUSDT 0.2 tensor(1.0716) 38 days 47.64% correct guesses
LINKUSDT 0.2 tensor(1.1205) 38 days 50.21% correct guesses
MEMEUSDT 0.2 tensor(1.2051) 38 days 50.21% correct guesses
MNTUSDT 0.2 tensor(1.0192) 

Epoch 100 Loss: 0.0005329972482286394


Epoch 200 Loss: 0.0005957674002274871
New best sum of logs 6.512720108032227


Epoch 300 Loss: 0.00036803146940656006


...

Epoch 45500 Loss: 1.5142003576329444e-05
ADAUSDT 0.2 tensor(1.1478) 38 days 45.92% correct guesses
APTUSDT 0.2 tensor(1.6476) 39 days 55.13% correct guesses
AVAXUSDT 0.2 tensor(1.0459) 38 days 48.50% correct guesses
BANANAUSDT 0.2 tensor(1.6624) 8 days 56.60% correct guesses
BATUSDT 0.2 tensor(1.4653) 38 days 54.08% correct guesses
BLZUSDT 0.2 tensor(1.2792) 38 days 53.22% correct guesses
BNBUSDT 0.2 tensor(1.0515) 38 days 49.79% correct guesses
BRETTUSDT 0.2 tensor(0.5251) 38 days 47.64% correct guesses
BSVUSDT 0.2 tensor(1.6088) 38 days 55.36% correct guesses
BTCUSDT 0.2 tensor(1.0847) 38 days 51.50% correct guesses
C98USDT 0.2 tensor(0.9388) 39 days 49.57% correct guesses
CEEKUSDT 0.2 tensor(0.8765) 38 days 44.21% correct guesses
DOGEUSDT 0.2 tensor(1.0930) 39 days 55.56% correct guesses
ETHUSDT 0.2 tensor(1.0523) 38 days 44.64% correct guesses
LINKUSDT 0.2 tensor(1.8255) 38 days 53.22% correct guesses
MEMEUSDT 0.2 tensor(0.7365) 38 days 47.64% correct guesses
MNTUSDT 0.2 tensor(1.0

Epoch 45600 Loss: 1.6945488823694177e-05


Epoch 45700 Loss: 1.4381194887391757e-05


Epoch 45800 Loss: 1.6724819943192415e-05


Epoch 45900 Loss: 1.3113747627357952e-05


#### save model

In [1]:
torch.save(model.state_dict(), f"models/VariationalRatesOfChange_{wandb.run.name}.pth")

NameError: name 'torch' is not defined